###  importing relevant libraries

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from collections import Counter
from tabulate import tabulate
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
import warnings
import sklearn
import time
import os
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix

time.sleep(18000)

### Discovering Labels

In [6]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    df["Label"]=df["Label"].replace(renamer)
    df=df.dropna()
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [7]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("Tthe folder could not be created!")

In [8]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [9]:
renamer={}

In [10]:
altime=0
#def most_frequent(List): 
#    return max(set(List), key = List.count) 


 
def most_frequent(List):
    occurence_count = Counter(List)
    occurence_count={k: v for k, v in sorted(occurence_count.items(), key=lambda item: item[1],reverse=True)}
    big=list(occurence_count.values())
    big=big.count(big[0])
    return list(occurence_count.keys())[np.random.randint(big)]


def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))
def create_exception(df): 
    exception_list=[]
    dominant_mac=[]
    for i in df['aggregated'].unique():
        k=df[df['aggregated']==i]
        for ii in ['MAC']:
            hist = {}
            for x in k[ii].values:
                hist[x] = hist.get(x, 0) + 1
            hist=dict(sorted(hist.items(), key=lambda item: item[1],reverse=True))
            temp=next(iter(hist))
            if temp not in dominant_mac:
                dominant_mac.append(temp)
            else:
                exception_list.append(temp)
    return exception_list




def merged(m_test,predict,step,mixed):
    second=time.time()
    mac_test=[]
    for q in m_test.index:
        mac_test.append(m_test[q])

    d_list=sorted(list(m_test.unique()))
    devices={}
    for q in d_list:
        devices[q]=[]    


    new_y=[0]*len(m_test)

    for q,qq in enumerate (mac_test):
        devices[qq].append(q)
    for q in devices:
        a = [devices[q][j:j + step] for j in range(0, len(devices[q]), step)]  
        for qq in a:
            step_list=[]
            for qqq in qq:
                step_list.append(predict[qqq])
            add=most_frequent(list(step_list))
            for qqq in qq:
                new_y[qqq]=add
    results=pd.DataFrame(m_test)
    results["aggregated"]=new_y
    results["normal"]=predict
    
    #MIXED METHOD
    if mixed:
        exception=create_exception(results)
        for q in exception:
            results.loc[results.MAC == q, 'aggregated'] = results['normal']

    return results["aggregated"].values,time.time()-second

## Calculation of evaluations

In [11]:
def score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ml):
    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,i,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time,altime]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return line,class_based_results

In [12]:


def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif
 
# Driver Code
#li1 = [10, 15, 20, 25, 30, 35, 40]
#li2 = [25, 40, 35]


In [13]:
def ML(loop1,loop2,output_csv1,cols,step,mixed,dname):

    
    for ii in ml_list:

        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        if ii in ["SVM","LR"]:
            repetition=1
        else:
            repetition=5
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):



            #TRAIN
            df = pd.read_csv(loop1,usecols=cols)
            df=df.dropna()
            df["Label"]=df["Label"].replace(renamer)
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  
            
            

            #TEST
            df = pd.read_csv(loop2,usecols=cols)
            df=df.dropna()
            df["Label"]=df["Label"].replace(renamer)
            df = shuffle(df, random_state=42)

            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes


            results_y=[]
            cv+=1
            results_y.append(y_test)


            #machine learning algorithm is applied in this section
            clf = ml_list[ii]#choose algorithm from ml_list dictionary
            second=time.time()
            clf.fit(X_train, y_train)
            train_time=(float((time.time()-second)) )
            second=time.time()
            predict =clf.predict(X_test)
            test_time=(float((time.time()-second)) )
            if step==1:
                altime=0
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
            else:
                predict,altime=merged(m_test,predict,step,mixed)
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                
                
            lines.append(line)
            df_cm = pd.DataFrame(confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
            
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            cm.to_csv(output_csv.replace(".csv","CM.csv"))
            #graph_name=output_csv+"_confusion matrix.pdf"   
            #plt.figure(figsize = (40,28))
            #sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            #plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()

            
            
            
            
            



# Machine learning applications 

In [14]:
feature_list=feature=col= ['pck_size', 'Ether_type', 'LLC_ctrl', 'EAPOL_version', 'EAPOL_type', 'IP_ihl', 'IP_tos', 'IP_len', 'IP_flags', 'IP_DF', 'IP_ttl', 'IP_options', 'ICMP_code', 'TCP_dataofs', 'TCP_FIN', 'TCP_ACK', 'TCP_window', 'UDP_len', 'DHCP_options', 'BOOTP_hlen', 'BOOTP_flags', 'BOOTP_sname', 'BOOTP_file', 'BOOTP_options', 'DNS_qr', 'DNS_rd', 'DNS_qdcount', 'dport_class', 'payload_bytes', 'entropy',
#"MAC",
'Label']

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost

In [16]:

ml_list={"DT":DecisionTreeClassifier(criterion="entropy",max_depth=19,max_features=24,min_samples_split=5 ),                
"RF":RandomForestClassifier(bootstrap=False,criterion="gini",max_depth=31,max_features=10,min_samples_split=2,n_estimators=136),           
"XGB":xgboost.XGBClassifier(n_estimators=1500,min_child_weight=1,max_depth=3,learning_rate=0.15),        
"KNN":KNeighborsClassifier(algorithm="ball_tree",leaf_size=35,n_neighbors=13,weights="distance"),         
"NB":GaussianNB(var_smoothing=2.31013e-06), 

"LR":LogisticRegression(C= 0.689136 ,penalty="l1",solver="liblinear" ),"SVM":SVC(gamma=1,C=10)}



In [17]:
find_the_way("./","clean")

[]

In [18]:
file_list={"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD11T":['./small/DI-S1.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD12":['./small/AD-S1.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD12T":['./small/DI-S2.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD22":['./small/AD-S2.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD22T":['./small/DI-S2.csv','./small/AD-S2.csv','./small/AD-S1.csv'],
"DD21":['./small/AD-S2.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD21T":['./small/DI-S1.csv','./small/AD-S2.csv','./small/AD-S1.csv']}
#file_list={"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv']}

In [19]:
say=0
dataset="./results/other/1/"
folder(dataset)

for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=1
        sayac=1
        output_csv=f"{dataset}/ML@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
        dname=(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
        print(dname)
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,dname)   



AD-S1-DI-S1
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S1    0     1  DT    0.669    0.687   0.678  0.687  0.643  0.65       0    0.532     0.063          0
 1  AD-S1-DI-S1    1     2  DT    0.703    0.715   0.73   0.715  0.678  0.686      0    0.513     0.047          0
 2  AD-S1-DI-S1    2     3  DT    0.684    0.702   0.695  0.702  0.66   0.666      0    0.534     0.058          0
 3  AD-S1-DI-S1    3     4  DT    0.684    0.702   0.702  0.702  0.663  0.666      0    0.533     0.061          0
 4  AD-S1-DI-S1    4     5  DT    0.677    0.693   0.7    0.693  0.65   0.658      0    0.536     0.06           0

                                  precision    recall    f1-score        support
------------------------------  -----------  --------  ----------  -------------
Amazon Echo        

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S2    0     1  NB    0.415    0.396   0.401  0.396  0.363  0.384      0    0.061     0.589          0
 1  AD-S1-DI-S2    1     2  NB    0.415    0.396   0.401  0.396  0.363  0.384      0    0.063     0.598          0
 2  AD-S1-DI-S2    2     3  NB    0.415    0.396   0.401  0.396  0.363  0.384      0    0.069     0.59           0
 3  AD-S1-DI-S2    3     4  NB    0.415    0.396   0.401  0.396  0.363  0.384      0    0.061     0.591          0
 4  AD-S1-DI-S2    4     5  NB    0.415    0.396   0.401  0.396  0.363  0.384      0    0.069     0.579          0

                                  precision      recall    f1-score       support
------------------------------  -----------  ----------  ----------  ------------
Amazon Echo                   

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S2-AD-S1    0     1  DT    0.694    0.712   0.757  0.712  0.682  0.677      0    0.584     0.02           0
 1  DI-S2-AD-S1    1     2  DT    0.728    0.74    0.749  0.74   0.723  0.713      0    0.502     0.023          0
 2  DI-S2-AD-S1    2     3  DT    0.724    0.752   0.791  0.752  0.745  0.709      0    0.501     0.013          0
 3  DI-S2-AD-S1    3     4  DT    0.675    0.686   0.74   0.686  0.645  0.657      0    0.602     0.016          0
 4  DI-S2-AD-S1    4     5  DT    0.797    0.81    0.796  0.81   0.782  0.786      0    0.496     0.02           0

                                  precision    recall    f1-score      support
------------------------------  -----------  --------  ----------  -----------
Amazon Echo                        0

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S2-AD-S1    0     1  NB    0.355    0.336   0.291  0.336  0.275   0.32      0    0.061     0.505          0
 1  DI-S2-AD-S1    1     2  NB    0.355    0.336   0.291  0.336  0.275   0.32      0    0.062     0.512          0
 2  DI-S2-AD-S1    2     3  NB    0.355    0.336   0.291  0.336  0.275   0.32      0    0.063     0.527          0
 3  DI-S2-AD-S1    3     4  NB    0.355    0.336   0.291  0.336  0.275   0.32      0    0.069     0.506          0
 4  DI-S2-AD-S1    4     5  NB    0.355    0.336   0.291  0.336  0.275   0.32      0    0.068     0.516          0

                                  precision      recall    f1-score       support
------------------------------  -----------  ----------  ----------  ------------
Amazon Echo                   

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S2-DI-S2    0     1  DT    0.663    0.691   0.64   0.691  0.623  0.646      0    0.441     0.02           0
 1  AD-S2-DI-S2    1     2  DT    0.631    0.639   0.693  0.639  0.593  0.612      0    0.486     0.02           0
 2  AD-S2-DI-S2    2     3  DT    0.662    0.69    0.742  0.69   0.641  0.644      0    0.462     0.014          0
 3  AD-S2-DI-S2    3     4  DT    0.713    0.715   0.746  0.715  0.667  0.698      0    0.435     0.026          0
 4  AD-S2-DI-S2    4     5  DT    0.583    0.586   0.667  0.586  0.537  0.561      0    0.374     0.02           0

                                  precision    recall    f1-score      support
------------------------------  -----------  --------  ----------  -----------
Amazon Echo                       0.

...

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S2-AD-S2    0     1  LR    0.635    0.651    0.69  0.651  0.647  0.615      0  347.422     0.018          0
 1  DI-S2-AD-S2    1     2  LR    0.635    0.651    0.69  0.651  0.647  0.615      0  346.939     0.028          0
 2  DI-S2-AD-S2    2     3  LR    0.635    0.651    0.69  0.651  0.647  0.615      0  348.928     0.031          0
 3  DI-S2-AD-S2    3     4  LR    0.635    0.651    0.69  0.651  0.647  0.615      0  347.033     0.022          0
 4  DI-S2-AD-S2    4     5  LR    0.635    0.651    0.69  0.651  0.647  0.615      0  346.064     0.031          0

                                  precision     recall    f1-score      support
------------------------------  -----------  ---------  ----------  -----------
Amazon Echo                       

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S2-DI-S1    0     1  RF    0.809    0.822   0.845  0.822  0.785  0.799      0    7.012     1.253          0
 1  AD-S2-DI-S1    1     2  RF    0.811    0.823   0.844  0.823  0.785  0.801      0    6.919     1.207          0
 2  AD-S2-DI-S1    2     3  RF    0.804    0.818   0.838  0.818  0.779  0.794      0    6.947     1.197          0
 3  AD-S2-DI-S1    3     4  RF    0.8      0.816   0.835  0.816  0.775  0.789      0    6.886     1.217          0
 4  AD-S2-DI-S1    4     5  RF    0.799    0.814   0.836  0.814  0.775  0.789      0    7.428     1.223          0

                                  precision     recall    f1-score       support
------------------------------  -----------  ---------  ----------  ------------
Amazon Echo                     

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S2-DI-S1    0     1  NB    0.433    0.422    0.36  0.422  0.372  0.403      0    0.061     0.516          0
 1  AD-S2-DI-S1    1     2  NB    0.433    0.422    0.36  0.422  0.372  0.403      0    0.063     0.516          0
 2  AD-S2-DI-S1    2     3  NB    0.433    0.422    0.36  0.422  0.372  0.403      0    0.053     0.536          0
 3  AD-S2-DI-S1    3     4  NB    0.433    0.422    0.36  0.422  0.372  0.403      0    0.065     0.536          0
 4  AD-S2-DI-S1    4     5  NB    0.433    0.422    0.36  0.422  0.372  0.403      0    0.069     0.537          0

                                  precision     recall    f1-score       support
------------------------------  -----------  ---------  ----------  ------------
Amazon Echo                     

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S1-AD-S2    0     1  DT    0.656    0.682   0.723  0.682  0.678  0.636      0    0.519     0.02           0
 1  DI-S1-AD-S2    1     2  DT    0.613    0.638   0.637  0.638  0.599  0.592      0    0.581     0.024          0
 2  DI-S1-AD-S2    2     3  DT    0.717    0.742   0.739  0.742  0.706  0.702      0    0.527     0.02           0
 3  DI-S1-AD-S2    3     4  DT    0.737    0.744   0.722  0.744  0.714  0.723      0    0.485     0.02           0
 4  DI-S1-AD-S2    4     5  DT    0.707    0.73    0.791  0.73   0.735  0.691      0    0.539     0.018          0

                                  precision    recall    f1-score       support
------------------------------  -----------  --------  ----------  ------------
Amazon Echo                       

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S1-AD-S2    0     1  NB    0.447     0.47   0.562   0.47  0.424  0.418      0    0.071     0.495          0
 1  DI-S1-AD-S2    1     2  NB    0.447     0.47   0.562   0.47  0.424  0.418      0    0.071     0.486          0
 2  DI-S1-AD-S2    2     3  NB    0.447     0.47   0.562   0.47  0.424  0.418      0    0.071     0.495          0
 3  DI-S1-AD-S2    3     4  NB    0.447     0.47   0.562   0.47  0.424  0.418      0    0.069     0.497          0
 4  DI-S1-AD-S2    4     5  NB    0.447     0.47   0.562   0.47  0.424  0.418      0    0.071     0.516          0

                                  precision      recall    f1-score      support
------------------------------  -----------  ----------  ----------  -----------
Amazon Echo                     

In [20]:
#!shutdown /s /t 360

# MLP

In [21]:
import sklearn
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
def create_model(params):
    model = Sequential()

    # Giriş katmanı
    model.add(Dense(units=params['units_input'], activation='relu', input_shape=(input_shape,)))

    # Gizli katmanlar
    for i in range(params['num_layers']):
        model.add(Dense(units=params[f'units_{i}'], activation='relu'))
        model.add(Dropout(params[f'dropout_{i}']))

    # Çıkış katmanı
    model.add(Dense(num_classes, activation='softmax'))

    # Optimizasyon ve kayıp fonksiyonunun belirlenmesi
    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

best_params = {'units_input': 32, 'num_layers': 2, 'units_0': 128, 'dropout_0': 0.0, 'learning_rate': 0.01, 'units_1': 32, 'dropout_1': 0.30000000000000004, 'units_2': 32, 'dropout_2': 0.2}
say=0
dataset="./results/other/1/"
folder(dataset)

for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=1
        sayac=1
        output_csv=f"{dataset}/MLP@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
        print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
        target_names=target_name(test)


        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
    
        repetition=5
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]


    
        for i in range(repetition):

            df = pd.read_csv(train,usecols=feature)
            df=df.dropna()
            df["Label"]=df["Label"].replace(renamer)
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  
            
            

            #TEST
            df = pd.read_csv(test,usecols=feature)
            df=df.dropna()
            df["Label"]=df["Label"].replace(feature)
            df = shuffle(df, random_state=42)

            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes
            
            y_train = to_categorical(y_train)
            #y_test = to_categorical(y_test)
            min_max_scaler = preprocessing.MinMaxScaler()
            X_train = min_max_scaler.fit_transform(X_train)
            X_test = min_max_scaler.fit_transform(X_test)
            
            input_shape=X_train.shape[1]
            num_classes=len(y_test.unique())
            # Modelin oluşturulması

            results_y=[]
            cv+=1
            results_y.append(y_test)

            model = create_model(best_params)
            #model.summary()
            second=time.time()
            model = create_model(best_params)
            model.fit(X_train, y_train, epochs=10, batch_size=32,validation_split=0.3,verbose= 0)#validation_data=(X_test, y_test)) #validation_split=0.3)
            train_time=(float((time.time()-second)) )
            second=time.time()
            
            predict =model.predict(X_test)
            predict = np.argmax(predict, axis=1)
            test_time=(float((time.time()-second)) )
            
            
            altime=0
            dname=f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}"
            line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,"MLP")
            
                
                
              
            lines.append(line)
            df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
                        
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv,index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            cm.to_csv(output_csv.replace(".csv","CM.csv"))
            #graph_name=output_csv+"_confusion matrix.pdf"   
            #plt.figure(figsize = (40,28))
            #sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            #plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()
  


AD-S1-DI-S1
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S1    0     1  MLP   0.365    0.315   0.312  0.315  0.275  0.327      0   97.305    13.099          0
 1  AD-S1-DI-S1    1     2  MLP   0.487    0.449   0.371  0.449  0.369  0.456      0  145.704    11.659          0
 2  AD-S1-DI-S1    2     3  MLP   0.431    0.402   0.354  0.402  0.333  0.397      0  124.49     12.103          0
 3  AD-S1-DI-S1    3     4  MLP   0.455    0.429   0.405  0.429  0.392  0.422      0  101.365    11.088          0
 4  AD-S1-DI-S1    4     5  MLP   0.454    0.418   0.353  0.418  0.344  0.422    

# CNN

In [22]:
import sklearn
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing

def create_model(params):
    model = Sequential()

    # Convolutional layer
    model.add(Conv1D(filters=params['conv1d_filters'], kernel_size=params['conv1d_kernel_size'], activation='relu', input_shape=((X_train.shape[1], X_train.shape[2]))))

    # Pooling layer
    model.add(MaxPooling1D(pool_size=params['max_pooling1d_pool_size']))

    # Flatten layer
    model.add(Flatten())

    # Dense layers
    model.add(Dense(units=params['dense_units_1'], activation='relu'))
    model.add(Dropout(params['dropout_rate_1']))
    model.add(Dense(units=params['dense_units_2'], activation='relu'))
   # model.add(Dropout(params['dropout_rate_2']))
    model.add(Dense(units=21, activation='softmax'))

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

best_params = {
    "conv1d_filters": 32,
    "conv1d_kernel_size": 3,
    "max_pooling1d_pool_size": 2,
    "dense_units_1": 96,
    "dropout_rate_1": 0.0,
    "dense_units_2": 21,
    "learning_rate": 0.01
}



say = 0
dataset = "./results/other/1/"
folder(dataset)

for ii in file_list:
    print()
    say += 1
    train = file_list[ii][0]
    test = file_list[ii][1]
    mixed = False
    step = 1
    sayac = 1
    output_csv = f"{dataset}/CNN@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
    print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
    target_names = target_name(test)

    class_based_results = pd.DataFrame()
    cm = pd.DataFrame()
    cv = 0
    repetition = 5
    lines = [["Dataset", "T", "CV", "ML", "Acc", "b_Acc", "Prec", "Rec", "F1", "kap", "ROC", "tra-T", "test-T", "Al-Time"]]

    for i in range(repetition):
        df = pd.read_csv(train, usecols=feature)
        df = df.dropna()
        df["Label"] = df["Label"].replace(renamer)
        X_train = df[df.columns[0:-1]]
        X_train = np.array(X_train)
        df[df.columns[-1]] = df[df.columns[-1]].astype('category')
        y_train = df[df.columns[-1]].cat.codes  

        df = pd.read_csv(test, usecols=feature)
        df = df.dropna()
        df["Label"] = df["Label"].replace(feature)
        df = shuffle(df, random_state=42)

        X_test = df[df.columns[0:-1]]
        X_test = np.array(X_test)
        df[df.columns[-1]] = df[df.columns[-1]].astype('category')
        y_test = df[df.columns[-1]].cat.codes

        y_train = to_categorical(y_train)
        min_max_scaler = preprocessing.MinMaxScaler()
        X_train = min_max_scaler.fit_transform(X_train)
        X_test = min_max_scaler.fit_transform(X_test)
        # Veriyi CNN modeline uygun hale getirme
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        
        input_shape = X_train.shape[1]
        num_classes = len(y_test.unique())

        results_y = []
        cv += 1
        results_y.append(y_test)

        model = create_model(best_params)
        second = time.time()
        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.3, verbose=0)
        train_time = (float((time.time() - second)))
        second = time.time()

        predict = model.predict(X_test)
        predict = np.argmax(predict, axis=1)
        test_time = (float((time.time() - second)))

        altime = 0
        dname = f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}"
        line, class_based_results = score(altime, train_time, test_time, predict, y_test, class_based_results, i, cv, dname, "CNN")

        lines.append(line)
        df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
        if cm.empty:
            cm = df_cm
        else:
            cm = cm.add(df_cm, fill_value=0)

    class_based_results = class_based_results / repetition
    results = pd.DataFrame(lines[1:], columns=lines[0])
    results.to_csv(output_csv, index=False)
    results = results.round(3)
    print(tabulate(results, headers=list(results.columns)))
    print()

    print(tabulate(class_based_results, headers=list(class_based_results.columns)))
    class_based_results.to_csv(output_csv.replace(".csv", "CB.csv"))
    if True:
        cm = cm // repetition
        cm.to_csv(output_csv.replace(".csv", "CM.csv"))



AD-S1-DI-S1
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S1    0     1  CNN   0.497    0.461   0.455  0.461  0.421  0.466      0  177.782    14.315          0
 1  AD-S1-DI-S1    1     2  CNN   0.455    0.409   0.387  0.409  0.375  0.423      0  180.282    14.808          0
 2  AD-S1-DI-S1    2     3  CNN   0.346    0.33    0.274  0.33   0.258  0.309      0  187.857    14.65           0
 3  AD-S1-DI-S1    3     4  CNN   0.446    0.419   0.368  0.419  0.339  0.413      0  176.643    13.77           0
 4  AD-S1-DI-S1    4     5  CNN   0.392    0.367   0.372  0.367  0.321  0.356

# LSTM

In [23]:
import sklearn
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
import time

# Veri yükleme fonksiyonu
def load_data(file_path, feature):
    df = pd.read_csv(file_path, usecols=feature) 
    X = df.iloc[:, :-1]
    df['Label'] = df['Label'].astype('category')
    y = df['Label'].cat.codes  
    min_max_scaler = preprocessing.MinMaxScaler()
    X_scaled = min_max_scaler.fit_transform(X)
    X_reshaped = np.reshape(X_scaled, (X_scaled.shape[0], X_scaled.shape[1], 1))
    return X_reshaped, y

# Model oluşturma fonksiyonu
def create_model(params):
    model = Sequential()
    model.add(layers.LSTM(units=params['lstm_0_units'],
                          return_sequences=True,
                          input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(layers.Dropout(rate=params['dropout_1']))
    
    for i in range(1, params['num_layers']):
        model.add(layers.LSTM(units=params[f'lstm_{i}_units'], return_sequences=True))
        model.add(layers.Dropout(rate=params[f'dropout_{i+1}']))
    
    model.add(layers.LSTM(units=params['lstm_1_units']))
    model.add(layers.Dropout(rate=params['dropout_last']))
    
    model.add(layers.Dense(params['units_last'], activation='softmax'))
    return model

# Parametreler
params = {'units': 32, 'dropout_1': 0.2, 'num_layers': 1, 'lstm_0_units': 96, 'dropout_2': 0.2, 'units_last': 128, 'dropout_last': 0.2, 'learning_rate': 0.01, 'lstm_1_units': 96, 'dropout_3': 0.0, 'lstm_2_units': 64, 'dropout_4': 0.0}


# Diğer değişkenler ve döngü
say = 0
dataset = "./results/other/1/"
folder(dataset)

for ii in file_list:
    print()
    say += 1
    train = file_list[ii][0]
    test = file_list[ii][1]
    mixed = False
    step = 1
    sayac = 1
    output_csv = f"{dataset}/LSTM@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
    print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
    target_names = target_name(test)

    class_based_results = pd.DataFrame()
    cm = pd.DataFrame()
    cv = 0
    repetition = 5
    lines = [["Dataset", "T", "CV", "ML", "Acc", "b_Acc", "Prec", "Rec", "F1", "kap", "ROC", "tra-T", "test-T", "Al-Time"]]

    for i in range(repetition):
        X_train, y_train = load_data(file_list[ii][0], feature)
        X_test, y_test = load_data(file_list[ii][1], feature)
        
        model = create_model(params)
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=params['learning_rate']),
                      loss='sparse_categorical_crossentropy',
                      metrics=["accuracy"])
        
        second = time.time()
        model.fit(X_train, y_train, epochs=10, validation_split=0.3, verbose=0)
        train_time = (float((time.time() - second)))
        second = time.time()

        predict = model.predict(X_test)
        predict = np.argmax(predict, axis=1)
        test_time = (float((time.time() - second)))

        altime = 0
        dname = f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}"
        line, class_based_results = score(altime, train_time, test_time, predict, y_test, class_based_results, i, cv, dname, "LSTM")

        lines.append(line)
        df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
        if cm.empty:
            cm = df_cm
        else:
            cm = cm.add(df_cm, fill_value=0)

    class_based_results = class_based_results / repetition
    results = pd.DataFrame(lines[1:], columns=lines[0])
    results.to_csv(output_csv, index=False)
    results = results.round(3)
    print(tabulate(results, headers=list(results.columns)))
    print()

    print(tabulate(class_based_results, headers=list(class_based_results.columns)))
    class_based_results.to_csv(output_csv.replace(".csv", "CB.csv"))
    if True:
        cm = cm // repetition
        cm.to_csv(output_csv.replace(".csv", "CM.csv"))



AD-S1-DI-S1
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 85s 16ms/step
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 86s 16ms/step
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 87s 16ms/step
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 86s 16ms/step
5399/5399 ━━━━━━━━━━━━━━━━━━━━ 87s 16ms/step
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S1    0     0  LSTM  0.347    0.302   0.264  0.302  0.251  0.308      0  1713.32   142.168          0
 1  AD-S1-DI-S1    1     0  LSTM  0.25     0.211   0.175  0.211  0.172  0.205      0  1800.5    142.132          0
 2  AD-S1-DI-S1    2     0  LSTM  0.358    0.338   0.311  0.338  0.279  0.323      0  1857.85   142.165          0
 3  AD-S1-DI-S1    3     0  LSTM  0.393    0.336   0.3    0.336  0.286  0.358      0  1744.65    89.134          0
 4  AD-S1-DI-S1    4     0  LSTM  0.241    0.2     0.145  0.2    0.158  

KeyboardInterrupt: 

In [24]:
file_list={#"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
#"DD11T":['./small/DI-S1.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD12":['./small/AD-S1.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD12T":['./small/DI-S2.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD22":['./small/AD-S2.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD22T":['./small/DI-S2.csv','./small/AD-S2.csv','./small/AD-S1.csv'],
"DD21":['./small/AD-S2.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD21T":['./small/DI-S1.csv','./small/AD-S2.csv','./small/AD-S1.csv']}
#file_list={"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv']}

In [25]:
import sklearn
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
import time

# Veri yükleme fonksiyonu
def load_data(file_path, feature):
    df = pd.read_csv(file_path, usecols=feature) 
    X = df.iloc[:, :-1]
    df['Label'] = df['Label'].astype('category')
    y = df['Label'].cat.codes  
    min_max_scaler = preprocessing.MinMaxScaler()
    X_scaled = min_max_scaler.fit_transform(X)
    X_reshaped = np.reshape(X_scaled, (X_scaled.shape[0], X_scaled.shape[1], 1))
    return X_reshaped, y

# Model oluşturma fonksiyonu
def create_model(params):
    model = Sequential()
    model.add(layers.LSTM(units=params['lstm_0_units'],
                          return_sequences=True,
                          input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(layers.Dropout(rate=params['dropout_1']))
    
    for i in range(1, params['num_layers']):
        model.add(layers.LSTM(units=params[f'lstm_{i}_units'], return_sequences=True))
        model.add(layers.Dropout(rate=params[f'dropout_{i+1}']))
    
    model.add(layers.LSTM(units=params['lstm_1_units']))
    model.add(layers.Dropout(rate=params['dropout_last']))
    
    model.add(layers.Dense(params['units_last'], activation='softmax'))
    return model

# Parametreler
params = {'units': 32, 'dropout_1': 0.2, 'num_layers': 1, 'lstm_0_units': 96, 'dropout_2': 0.2, 'units_last': 128, 'dropout_last': 0.2, 'learning_rate': 0.01, 'lstm_1_units': 96, 'dropout_3': 0.0, 'lstm_2_units': 64, 'dropout_4': 0.0}


# Diğer değişkenler ve döngü
say = 0
dataset = "./results/other/1/"
folder(dataset)

for ii in file_list:
    print()
    say += 1
    train = file_list[ii][0]
    test = file_list[ii][1]
    mixed = False
    step = 1
    sayac = 1
    output_csv = f"{dataset}/LSTM@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
    print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
    target_names = target_name(test)

    class_based_results = pd.DataFrame()
    cm = pd.DataFrame()
    cv = 0
    repetition = 1
    lines = [["Dataset", "T", "CV", "ML", "Acc", "b_Acc", "Prec", "Rec", "F1", "kap", "ROC", "tra-T", "test-T", "Al-Time"]]

    for i in range(repetition):
        X_train, y_train = load_data(file_list[ii][0], feature)
        X_test, y_test = load_data(file_list[ii][1], feature)
        
        model = create_model(params)
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=params['learning_rate']),
                      loss='sparse_categorical_crossentropy',
                      metrics=["accuracy"])
        
        second = time.time()
        model.fit(X_train, y_train, epochs=10, validation_split=0.3, verbose=0)
        train_time = (float((time.time() - second)))
        second = time.time()

        predict = model.predict(X_test)
        predict = np.argmax(predict, axis=1)
        test_time = (float((time.time() - second)))

        altime = 0
        dname = f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}"
        line, class_based_results = score(altime, train_time, test_time, predict, y_test, class_based_results, i, cv, dname, "LSTM")

        lines.append(line)
        df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
        if cm.empty:
            cm = df_cm
        else:
            cm = cm.add(df_cm, fill_value=0)

    class_based_results = class_based_results / repetition
    results = pd.DataFrame(lines[1:], columns=lines[0])
    results.to_csv(output_csv, index=False)
    results = results.round(3)
    print(tabulate(results, headers=list(results.columns)))
    print()

    print(tabulate(class_based_results, headers=list(class_based_results.columns)))
    class_based_results.to_csv(output_csv.replace(".csv", "CB.csv"))
    if True:
        cm = cm // repetition
        cm.to_csv(output_csv.replace(".csv", "CM.csv"))



AD-S1-DI-S2
5380/5380 ━━━━━━━━━━━━━━━━━━━━ 96s 18ms/step
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S2    0     0  LSTM  0.419    0.385   0.343  0.385  0.324  0.384      0  2447.65   116.815          0

                                  precision    recall    f1-score       support
------------------------------  -----------  --------  ----------  ------------
Amazon Echo                       0.426386   0.5129     0.465659    10000
Awair air quality monitor         0.37076    0.9804     0.538046    10000
Belkin Wemo switch                0.352356   0.7619     0.481864    10000
Belkin wemo motion sensor         0.18184    0.3062     0.228175    10000
Dropcam                           0.956172   0.9992     0.977213    10000
HP Printer                        0.816213   0.1319     0.227101  